In [ ]:
# Análise dos 10 Logradouros - VERSÃO RAW COM GRÁFICOS
# Base: Planilha RAW (data/raw)
# Estrutura: Data, Equipe, Logradouro, Período, Quantidade
# Períodos: 10h (Manhã) e 15h (Tarde)
# Período: Solicitado ao usuário

# %% [markdown]
# # 1. Configuração Inicial

# %%
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from pathlib import Path
import warnings
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import matplotlib.dates as mdates

warnings.filterwarnings('ignore')

# Configurar estilo dos gráficos
plt.style.use('seaborn-v0_8-whitegrid')
sns.set_palette("husl")

print("✓ Bibliotecas importadas")
print(f"✓ Análise iniciada em: {datetime.now().strftime('%d/%m/%Y %H:%M:%S')}")

# %% [markdown]
# # 2. Definir Período de Análise

# %%
print("\n" + "=" * 80)
print("DEFINIR PERÍODO DO RELATÓRIO")
print("=" * 80)

data_inicio_str = input("\nDigite a DATA INICIAL (dd/mm/aaaa): ")
data_fim_str = input("Digite a DATA FINAL (dd/mm/aaaa): ")

# Converter datas
try:
    data_inicio_dt = datetime.strptime(data_inicio_str, "%d/%m/%Y")
    data_fim_dt = datetime.strptime(data_fim_str, "%d/%m/%Y")
    
    # Converter para formato string usado nos filtros
    data_inicio = data_inicio_dt.strftime('%Y-%m-%d')
    data_fim = data_fim_dt.strftime('%Y-%m-%d')
    
    print(f"\n✓ Período selecionado: {data_inicio_dt.strftime('%d/%m/%Y')} a {data_fim_dt.strftime('%d/%m/%Y')}")
    
    # Calcular quantidade de dias
    dias_periodo = (data_fim_dt - data_inicio_dt).days + 1
    total_periodos = dias_periodo * 2  # 2 períodos por dia (Manhã e Tarde)
    
    print(f"\n{'=' * 80}")
    print(f"Período: {data_inicio_dt.strftime('%d/%m/%Y')} a {data_fim_dt.strftime('%d/%m/%Y')}")
    print(f"  • Dias: {dias_periodo}")
    print(f"  • Períodos por dia: 2 (Manhã e Tarde)")
    print(f"  • Total de períodos: {total_periodos}")
    print(f"{'=' * 80}")
    
except ValueError:
    print("❌ Formato de data inválido! Use dd/mm/aaaa")
    raise Exception("Formato de data inválido")

# %% [markdown]
# # 3. Selecionar Arquivo RAW

# %%
from pathlib import Path

print("\n" + "=" * 80)
print("SELEÇÃO DO ARQUIVO RAW")
print("=" * 80)

# Detectar raiz do projeto
script_dir = Path(__file__).parent if '__file__' in globals() else Path.cwd()
if script_dir.name == 'notebooks':
    project_root = script_dir.parent
elif script_dir.name == 'etl':
    project_root = script_dir.parent.parent
else:
    project_root = script_dir

pasta_raw = project_root / 'data' / 'raw'
print(f"📂 Pasta raw: {pasta_raw}")

# Listar arquivos disponíveis
arquivos_disponiveis = sorted(list(pasta_raw.glob('*.xlsx')), 
                               key=lambda x: x.stat().st_mtime, 
                               reverse=True)

if arquivos_disponiveis:
    print(f"\n📁 Arquivos disponíveis (mais recentes primeiro):")
    for i, arq in enumerate(arquivos_disponiveis, 1):
        modificado = datetime.fromtimestamp(arq.stat().st_mtime).strftime('%d/%m/%Y %H:%M')
        print(f"  {i}. {arq.name}")
        print(f"     Modificado em: {modificado}\n")
    
    print("=" * 80)
    selecao = int(input("Digite o número do arquivo que deseja analisar: "))
    arquivo_selecionado = arquivos_disponiveis[selecao - 1]
    print(f"✓ Arquivo selecionado: {arquivo_selecionado.name}")
else:
    print(f"\n⚠️ Nenhum arquivo .xlsx encontrado em '{pasta_raw}'")
    raise FileNotFoundError(f"Nenhum arquivo em {pasta_raw}")

print("=" * 80)

# %% [markdown]
# # 4. Carregar Dados RAW

# %%
print("\n" + "=" * 80)
print("CARREGANDO DADOS RAW")
print("=" * 80)

try:
    df = pd.read_excel(arquivo_selecionado)
    print(f"✓ Arquivo carregado: {arquivo_selecionado.name}")
    print(f"✓ Total de registros: {len(df):,}")
    print(f"\nColunas disponíveis:")
    for col in df.columns:
        print(f"  - {col}")
    
    # Verificar campos necessários
    campos_necessarios = ['Data', 'Logradouro', 'Período', 'Quantidade']
    campos_faltando = [c for c in campos_necessarios if c not in df.columns]
    
    if campos_faltando:
        print(f"\n✗ ERRO: Campos faltando: {campos_faltando}")
        raise KeyError("Campos necessários não encontrados")
    
except Exception as e:
    print(f"✗ ERRO ao carregar arquivo: {str(e)}")
    raise

# %% [markdown]
# # 5. Preparar Dados

# %%
print("\n" + "=" * 80)
print("PREPARANDO DADOS")
print("=" * 80)

# Converter data
df['Data'] = pd.to_datetime(df['Data'], errors='coerce')

# Garantir que Quantidade seja numérico
df['Quantidade'] = pd.to_numeric(df['Quantidade'], errors='coerce').fillna(0)

# Limpar espaços em branco no Logradouro e Período
df['Logradouro'] = df['Logradouro'].astype(str).str.strip()
df['Período'] = df['Período'].astype(str).str.strip()

print(f"✓ Campo 'Data' convertido para datetime")
print(f"✓ Campo 'Quantidade' convertido para numérico")
print(f"✓ Campo 'Logradouro' limpo")
print(f"✓ Registros preparados: {len(df):,}")

# Extrair região do logradouro (tudo antes do primeiro " - ")
df['Região'] = df['Logradouro'].str.split(' - ', n=1).str[0].str.strip()

print(f"\nRegiões encontradas:")
regioes_unicas = sorted(df['Região'].unique())
for regiao in regioes_unicas:
    qtd_log = df[df['Região'] == regiao]['Logradouro'].nunique()
    print(f"  - {regiao} ({qtd_log} logradouros)")

# Verificar períodos únicos ANTES do mapeamento
print(f"\nPeríodos encontrados (originais):")
for periodo in sorted(df['Período'].unique()):
    print(f"  - {periodo}")

# Mapear períodos: 10h → Manhã, 15h → Tarde
mapeamento_periodos = {
    '10h': 'Manhã',
    '15h': 'Tarde'
}

df['Período_padrao'] = df['Período'].map(mapeamento_periodos)

# Verificar se há períodos não mapeados
periodos_nao_mapeados = df[df['Período_padrao'].isna()]['Período'].unique()
if len(periodos_nao_mapeados) > 0:
    print(f"\n⚠️ ATENÇÃO: Períodos não mapeados encontrados:")
    for periodo in periodos_nao_mapeados:
        print(f"  - '{periodo}'")
    print(f"\n   Esses registros serão ignorados na análise.")
    # Remover registros não mapeados
    df = df[df['Período_padrao'].notna()].copy()

print(f"\nPeríodos padronizados:")
for periodo in sorted(df['Período_padrao'].unique()):
    print(f"  - {periodo}")

print(f"\n✓ Registros após limpeza: {len(df):,}")

# %% [markdown]
# # 6. Filtrar Período

# %%
print("\n" + "=" * 80)
print("FILTRANDO PERÍODO")
print("=" * 80)

# Filtrar período
df_periodo = df[
    (df['Data'] >= data_inicio) & 
    (df['Data'] <= data_fim)
].copy()

print(f"Data início: {data_inicio}")
print(f"Data fim: {data_fim}")
print(f"Dias no período: {dias_periodo}")
print(f"✓ Registros no período: {len(df_periodo):,}")
print(f"  Logradouros únicos: {df_periodo['Logradouro'].nunique()}")
print(f"  Regiões únicas: {df_periodo['Região'].nunique()}")
print(f"  Períodos únicos: {sorted(df_periodo['Período_padrao'].unique())}")

# %% [markdown]
# # 7. Calcular Soma Total por Região (para ordenação)

# %%
print("\n" + "=" * 80)
print("CALCULANDO SOMA TOTAL POR REGIÃO")
print("=" * 80)

# Calcular soma total de pessoas por região
soma_por_regiao = df_periodo.groupby('Região')['Quantidade'].sum().sort_values(ascending=False)

print(f"✓ Regiões ordenadas por soma total:")
for regiao, soma in soma_por_regiao.items():
    print(f"  {regiao}: {soma:,.0f} pessoas")

# Lista ordenada de regiões (será usada para ordenar as abas)
regioes_ordenadas = soma_por_regiao.index.tolist()

# %% [markdown]
# # 8. Agrupar por Região e Logradouro

# %%
print("\n" + "=" * 80)
print("AGRUPANDO POR REGIÃO E LOGRADOURO")
print("=" * 80)

# Agrupar por Região e Logradouro
df_agrupado = df_periodo.groupby(['Região', 'Logradouro']).agg({
    'Quantidade': 'sum'
}).reset_index()

# Renomear coluna
df_agrupado.rename(columns={'Quantidade': 'Soma pessoas'}, inplace=True)

# Calcular Média pessoas (soma / total de períodos)
df_agrupado['Média pessoas'] = df_agrupado['Soma pessoas'] / total_periodos

# Para cada logradouro, calcular média por período
print("\n✓ Calculando médias por período para cada logradouro...")

medias_lista = []

for idx, row in df_agrupado.iterrows():
    regiao = row['Região']
    logradouro = row['Logradouro']
    
    # Filtrar dados desse logradouro
    df_log = df_periodo[df_periodo['Logradouro'] == logradouro].copy()
    
    # Agrupar por período e somar
    soma_por_periodo = df_log.groupby('Período_padrao')['Quantidade'].sum()
    
    # Calcular média (dividir pela quantidade de dias)
    media_manha = soma_por_periodo.get('Manhã', 0) / dias_periodo
    media_tarde = soma_por_periodo.get('Tarde', 0) / dias_periodo
    
    medias_lista.append({
        'Região': regiao,
        'Logradouro': logradouro,
        'Soma pessoas': row['Soma pessoas'],
        'Média pessoas': row['Média pessoas'],
        'Manhã': media_manha,
        'Tarde': media_tarde
    })

df_completo = pd.DataFrame(medias_lista)

print(f"✓ Agrupamento concluído")
print(f"  Total de logradouros: {len(df_completo)}")
print(f"  Total de regiões: {df_completo['Região'].nunique()}")

# %% [markdown]
# # 9. Criar Rankings por Região (Média)

# %%
print("\n" + "=" * 80)
print("CRIANDO RANKINGS DE MÉDIA POR REGIÃO")
print("=" * 80)

rankings_media = {}

for regiao in regioes_ordenadas:
    df_regiao = df_completo[df_completo['Região'] == regiao].copy()
    
    # Ordenar por Média pessoas (decrescente)
    df_regiao = df_regiao.sort_values('Média pessoas', ascending=False)
    
    rankings_media[regiao] = df_regiao
    
    print(f"\n✓ Região: {regiao}")
    print(f"  Total de logradouros: {len(df_regiao)}")
    print(f"  Maior média: {df_regiao['Média pessoas'].max():.2f}")
    print(f"  Menor média: {df_regiao['Média pessoas'].min():.2f}")

# %% [markdown]
# # 10. Criar Abas no Excel

# %%
def limpar_nome_aba(nome):
    """Remove caracteres inválidos para nomes de abas do Excel"""
    # Caracteres inválidos no Excel: : \ / ? * [ ]
    caracteres_invalidos = [':', '\\', '/', '?', '*', '[', ']']
    nome_limpo = nome
    for char in caracteres_invalidos:
        nome_limpo = nome_limpo.replace(char, '-')
    # Limitar a 31 caracteres
    return nome_limpo[:31]

print("\n" + "=" * 80)
print("CRIANDO ABAS NO EXCEL POR REGIÃO")
print("=" * 80)

# Caminho do arquivo de saída
pasta_docs = project_root / 'docs'
pasta_docs.mkdir(parents=True, exist_ok=True)
output_file = pasta_docs / 'top10_raw_analise_por_regiao.xlsx'

# Criar o arquivo Excel
with pd.ExcelWriter(output_file, engine='openpyxl') as writer:
    
    # Contador de abas para garantir nomes únicos
    nomes_abas_usados = {}
    
    # Para cada região (ordenadas por soma total)
    for regiao in regioes_ordenadas:
        
        print(f"\n{'=' * 80}")
        print(f"Processando Região: {regiao}")
        print(f"{'=' * 80}")
        
        # 1. ABA: Rank Média [Região]
        df_rank_media = rankings_media[regiao].copy()
        df_rank_media_exportar = df_rank_media[[
            'Logradouro', 
            'Soma pessoas', 
            'Média pessoas',
            'Manhã',
            'Tarde'
        ]].copy()
        df_rank_media_exportar.insert(0, '#', range(1, len(df_rank_media_exportar) + 1))
        
        # Nome da aba (limitar a 31 caracteres)
        nome_aba_media = f"Rank Média {regiao}"
        if len(nome_aba_media) > 31:
            nome_aba_media = f"Média {regiao}"[:31]
        
        # Limpar caracteres inválidos
        nome_aba_media = limpar_nome_aba(nome_aba_media)
        
        # Garantir nome único
        nome_aba_media_final = nome_aba_media
        contador = 1
        while nome_aba_media_final in nomes_abas_usados:
            nome_aba_media_final = f"{nome_aba_media[:28]}_{contador}"
            contador += 1
        nomes_abas_usados[nome_aba_media_final] = True
        
        df_rank_media_exportar.to_excel(writer, sheet_name=nome_aba_media_final, index=False)
        print(f"✓ Aba '{nome_aba_media_final}' criada ({len(df_rank_media_exportar)} logradouros)")
        
        # 2. ABAS INDIVIDUAIS: Para cada logradouro do Rank Média (NA ORDEM DO RANKING)
        logradouros_ordenados = df_rank_media['Logradouro'].tolist()
        
        for idx, logradouro in enumerate(logradouros_ordenados, 1):
            
            # Filtrar dados originais para esse logradouro
            df_detalhado = df_periodo[
                df_periodo['Logradouro'] == logradouro
            ].copy()
            
            # Criar pivot: transformar períodos em colunas
            df_pivot = df_detalhado.pivot_table(
                index='Data',
                columns='Período_padrao',
                values='Quantidade',
                aggfunc='sum',
                fill_value=0
            ).reset_index()
            
            # PREENCHER DATAS FALTANTES
            # Criar range completo de datas do período
            data_inicio_dt_range = pd.to_datetime(data_inicio)
            data_fim_dt_range = pd.to_datetime(data_fim)
            todas_datas = pd.date_range(start=data_inicio_dt_range, end=data_fim_dt_range, freq='D')
            
            # Criar DataFrame com todas as datas
            df_todas_datas = pd.DataFrame({'Data': todas_datas})
            
            # Fazer merge para incluir datas faltantes
            df_pivot = df_todas_datas.merge(df_pivot, on='Data', how='left')
            
            # Preencher valores faltantes com 0 (colunas de período)
            colunas_periodo = [col for col in df_pivot.columns if col != 'Data']
            df_pivot[colunas_periodo] = df_pivot[colunas_periodo].fillna(0)
            
            # Garantir que todas as colunas de período existam
            for periodo in ['Manhã', 'Tarde']:
                if periodo not in df_pivot.columns:
                    df_pivot[periodo] = 0
            
            # Preparar DataFrame para exportação na ordem correta
            df_exportar = pd.DataFrame()
            df_exportar['Logradouro'] = [logradouro] * len(df_pivot)
            df_exportar['Data'] = df_pivot['Data'].dt.strftime('%d/%m/%Y')
            df_exportar['Manhã'] = df_pivot['Manhã'].astype(int)
            df_exportar['Tarde'] = df_pivot['Tarde'].astype(int)
            
            # Nome da aba (limitado a 31 caracteres)
            # Extrair apenas o nome do logradouro (depois do " - ")
            if ' - ' in logradouro:
                nome_logr_curto = logradouro.split(' - ', 1)[1]
            else:
                nome_logr_curto = logradouro
            
            # Limpar caracteres inválidos
            nome_aba = limpar_nome_aba(nome_logr_curto)
            
            # Garantir nome único de aba
            nome_aba_final = nome_aba
            contador = 1
            while nome_aba_final in nomes_abas_usados:
                sufixo = f"_{contador}"
                nome_aba_final = nome_aba[:31-len(sufixo)] + sufixo
                contador += 1
            nomes_abas_usados[nome_aba_final] = True
            
            # Escrever aba
            df_exportar.to_excel(writer, sheet_name=nome_aba_final, index=False)
            print(f"  [{idx}/{len(logradouros_ordenados)}] ✓ '{nome_aba_final}' ({len(df_exportar)} dias)")

print(f"\n{'=' * 80}")
print(f"✓ Arquivo criado: {output_file}")
print(f"  Total de abas criadas: {len(nomes_abas_usados)}")
print(f"{'=' * 80}")

# %% [markdown]
# # 11. Resumo Executivo

# %%
print("\n" + "=" * 80)
print("RESUMO EXECUTIVO")
print("=" * 80)

# Contar total de logradouros
total_logradouros = df_completo['Logradouro'].nunique()
total_regioes = len(regioes_ordenadas)

# Resumo por região
resumo_regioes = []
for regiao in regioes_ordenadas:
    qtd_logradouros = len(rankings_media[regiao])
    soma_total = soma_por_regiao[regiao]
    
    resumo_regioes.append(f"""
  • {regiao}:
    - Logradouros: {qtd_logradouros}
    - Soma total: {soma_total:,.0f} pessoas""")

resumo_regioes_str = "\n".join(resumo_regioes)

print(f"""
ANÁLISE: RANKINGS POR REGIÃO (VERSÃO RAW)

ARQUIVO ANALISADO:
- {arquivo_selecionado.name}

PERÍODO:
- De: {data_inicio_dt.strftime('%d/%m/%Y')}
- Até: {data_fim_dt.strftime('%d/%m/%Y')}
- Total de dias: {dias_periodo}
- Total de períodos: {total_periodos} ({dias_periodo} dias × 2 períodos)

METODOLOGIA:
- Fonte: Planilha RAW (data/raw)
- Estrutura: Data, Equipe, Logradouro, Período, Quantidade
- Períodos: 10h (Manhã) e 15h (Tarde)
- Extração de Região: Tudo antes do primeiro " - "
- Agrupamento: Por Região e depois por Logradouro
- Ordenação das Regiões: Por soma total de pessoas (decrescente)

CÁLCULOS:
- Soma pessoas: Soma de Quantidade de todos os períodos
- Média pessoas: Soma pessoas ÷ {total_periodos} períodos
- Média por período: Soma do período ÷ {dias_periodo} dias

DADOS PROCESSADOS:
- Registros totais no período: {len(df_periodo):,}
- Regiões encontradas: {total_regioes}
- Logradouros únicos: {total_logradouros}

RESUMO POR REGIÃO:
{resumo_regioes_str}

ARQUIVO GERADO:
✓ {output_file}

ESTRUTURA DAS ABAS:
Para cada região (ordenadas por soma total):
  1. Rank Média [Região] - Todos os logradouros ordenados por média
  2. Abas individuais de cada logradouro (na ordem do rank, com TODAS as datas)

REGIÃO COM MAIOR SOMA TOTAL:
- Região: {regioes_ordenadas[0]}
- Soma total: {soma_por_regiao[regioes_ordenadas[0]]:,.0f} pessoas
- Logradouros: {len(rankings_media[regioes_ordenadas[0]])}
""")

# Top 1 de cada região
print(f"\nTOP 1 DE CADA REGIÃO (POR MÉDIA):")
print("=" * 80)

for regiao in regioes_ordenadas:
    df_regiao = rankings_media[regiao]
    if len(df_regiao) > 0:
        top1 = df_regiao.iloc[0]
        print(f"\n{regiao}:")
        print(f"  • Logradouro: {top1['Logradouro']}")
        print(f"  • Média pessoas: {top1['Média pessoas']:.2f}")
        print(f"  • Manhã: {top1['Manhã']:.2f} | Tarde: {top1['Tarde']:.2f}")

print("\n" + "=" * 80)
print("✓ ANÁLISE CONCLUÍDA!")
print(f"✓ {datetime.now().strftime('%d/%m/%Y %H:%M:%S')}")
print("=" * 80)

# %% [markdown]
# # 12. Geração de Gráficos - Evolução Temporal

# %%
print("\n" + "=" * 80)
print("GERANDO GRÁFICOS DE EVOLUÇÃO TEMPORAL")
print("=" * 80)

# Criar pastas para salvar gráficos - ESTRUTURA CORRETA
pasta_graficos = project_root / 'docs' / 'graficos'
pasta_complexo = pasta_graficos / 'complexo_okuhara'
pasta_graficos_individuais = pasta_complexo / 'logradouros_individuais'
pasta_graficos_regioes = pasta_complexo / 'regioes'

pasta_graficos.mkdir(parents=True, exist_ok=True)
pasta_complexo.mkdir(parents=True, exist_ok=True)
pasta_graficos_individuais.mkdir(parents=True, exist_ok=True)
pasta_graficos_regioes.mkdir(parents=True, exist_ok=True)

print(f"📁 Pasta principal: {pasta_graficos}")
print(f"📁 Pasta complexo: {pasta_complexo}")
print(f"📁 Pasta individuais: {pasta_graficos_individuais}")
print(f"📁 Pasta regiões: {pasta_graficos_regioes}")

# %%
def calcular_tendencia(datas, valores):
    """
    Calcula linha de tendência linear
    Retorna valores da tendência para plotagem e variação absoluta
    """
    # Converter datas para números (dias desde a primeira data)
    datas_num = [(d - datas.min()).days for d in datas]
    
    # Calcular regressão linear
    slope, intercept, r_value, p_value, std_err = stats.linregress(datas_num, valores)
    
    # Calcular valores da linha de tendência
    tendencia = [slope * x + intercept for x in datas_num]
    
    # Calcular variação ABSOLUTA do início ao fim (diferença no eixo Y)
    if len(tendencia) > 0:
        valor_inicio = tendencia[0]
        valor_fim = tendencia[-1]
        variacao_absoluta = valor_fim - valor_inicio
    else:
        variacao_absoluta = 0
    
    return tendencia, slope, r_value**2, variacao_absoluta

def limpar_nome_arquivo(nome):
    """
    Remove caracteres problemáticos para nomes de arquivos
    """
    # Caracteres problemáticos: : \ / ? * " < > |
    caracteres_problematicos = [':', '\\', '/', '?', '*', '"', '<', '>', '|']
    nome_limpo = nome
    for char in caracteres_problematicos:
        nome_limpo = nome_limpo.replace(char, '-')
    return nome_limpo

print("✓ Funções auxiliares definidas")

# %%
# CALCULAR VALOR MÁXIMO GLOBAL PARA PADRONIZAÇÃO DOS EIXOS (INDIVIDUAIS)
print("\n" + "=" * 80)
print("CALCULANDO VALOR MÁXIMO GLOBAL (GRÁFICOS INDIVIDUAIS)")
print("=" * 80)

# Encontrar o maior valor entre todos os logradouros (Manhã e Tarde)
valor_max_global_individuais = 0

for regiao in regioes_ordenadas:
    logradouros_regiao = rankings_media[regiao]['Logradouro'].tolist()
    
    for logradouro in logradouros_regiao:
        df_log = df_periodo[df_periodo['Logradouro'] == logradouro].copy()
        
        if len(df_log) > 0:
            max_manha = df_log[df_log['Período_padrao'] == 'Manhã']['Quantidade'].max()
            max_tarde = df_log[df_log['Período_padrao'] == 'Tarde']['Quantidade'].max()
            valor_max_logradouro = max(max_manha, max_tarde)
            
            if valor_max_logradouro > valor_max_global_individuais:
                valor_max_global_individuais = valor_max_logradouro

# Adicionar margem de 10% para melhor visualização
valor_max_global_individuais = int(valor_max_global_individuais * 1.1)

print(f"✓ Valor máximo global (individuais): {valor_max_global_individuais}")
print(f"  Este será o limite superior do eixo Y em todos os gráficos individuais")
print("=" * 80)

# %%
# CALCULAR VALOR MÁXIMO GLOBAL PARA PADRONIZAÇÃO DOS EIXOS (REGIÕES)
print("\n" + "=" * 80)
print("CALCULANDO VALOR MÁXIMO GLOBAL (GRÁFICOS DE REGIÕES)")
print("=" * 80)

# Encontrar o maior valor entre todas as médias de regiões
valor_max_global_regioes = 0

for regiao in regioes_ordenadas:
    df_regiao = rankings_media[regiao].copy()
    max_manha = df_regiao['Manhã'].max()
    max_tarde = df_regiao['Tarde'].max()
    valor_max_regiao = max(max_manha, max_tarde)
    
    if valor_max_regiao > valor_max_global_regioes:
        valor_max_global_regioes = valor_max_regiao

# Arredondar e adicionar margem de 10%
valor_max_global_regioes = int(valor_max_global_regioes * 1.1)

print(f"✓ Valor máximo global (regiões): {valor_max_global_regioes}")
print(f"  Este será o limite superior do eixo X em todos os gráficos de regiões")
print("=" * 80)

# %% [markdown]
# ## 12.1. Gráficos Individuais de Evolução por Logradouro

# %%
print("\n" + "=" * 80)
print("GERANDO GRÁFICOS INDIVIDUAIS POR LOGRADOURO")
print("=" * 80)

total_graficos = 0

for regiao in regioes_ordenadas:
    print(f"\n{'=' * 80}")
    print(f"📊 Região: {regiao}")
    print(f"{'=' * 80}")
    
    # Pegar logradouros da região (ordenados por média)
    logradouros_regiao = rankings_media[regiao]['Logradouro'].tolist()
    
    for idx, logradouro in enumerate(logradouros_regiao, 1):
        
        # Filtrar dados do logradouro
        df_log = df_periodo[df_periodo['Logradouro'] == logradouro].copy()
        
        # Criar pivot com TODAS as datas do período
        data_inicio_dt_range = pd.to_datetime(data_inicio)
        data_fim_dt_range = pd.to_datetime(data_fim)
        todas_datas = pd.date_range(start=data_inicio_dt_range, end=data_fim_dt_range, freq='D')
        
        df_todas_datas = pd.DataFrame({'Data': todas_datas})
        
        df_pivot = df_log.pivot_table(
            index='Data',
            columns='Período_padrao',
            values='Quantidade',
            aggfunc='sum',
            fill_value=0
        ).reset_index()
        
        # Merge para incluir todas as datas
        df_pivot = df_todas_datas.merge(df_pivot, on='Data', how='left')
        
        # Preencher valores faltantes com 0
        for periodo in ['Manhã', 'Tarde']:
            if periodo not in df_pivot.columns:
                df_pivot[periodo] = 0
            else:
                df_pivot[periodo] = df_pivot[periodo].fillna(0)
        
        # ARREDONDAR OS DADOS ANTES DE PLOTAR
        df_pivot['Manhã'] = df_pivot['Manhã'].round(0).astype(int)
        df_pivot['Tarde'] = df_pivot['Tarde'].round(0).astype(int)
        
        # Criar o gráfico
        fig, ax = plt.subplots(figsize=(16, 9))
        
        # Plotar linhas de dados - NOVAS CORES
        line_manha = ax.plot(df_pivot['Data'], df_pivot['Manhã'], 
                marker='o', linewidth=2.5, markersize=7, 
                label='Manhã (10h)', color='#76B6FF', alpha=0.9, zorder=3)
        
        line_tarde = ax.plot(df_pivot['Data'], df_pivot['Tarde'], 
                marker='s', linewidth=2.5, markersize=7, 
                label='Tarde (15h)', color='#E67E22', alpha=0.9, zorder=3)
        
        # Adicionar rótulos de dados nas linhas - NOVAS CORES
        for i, (data, valor) in enumerate(zip(df_pivot['Data'], df_pivot['Manhã'])):
            if valor > 0:  # Só mostrar se houver valor
                ax.annotate(f'{valor}', 
                           xy=(data, valor), 
                           xytext=(0, 8),
                           textcoords='offset points',
                           ha='center', va='bottom',
                           fontsize=8, color='#76B6FF',
                           fontweight='bold')
        
        for i, (data, valor) in enumerate(zip(df_pivot['Data'], df_pivot['Tarde'])):
            if valor > 0:  # Só mostrar se houver valor
                ax.annotate(f'{valor}', 
                           xy=(data, valor), 
                           xytext=(0, -12),
                           textcoords='offset points',
                           ha='center', va='top',
                           fontsize=8, color='#E67E22',
                           fontweight='bold')
        
        # Calcular e plotar linhas de tendência COM VARIAÇÃO ABSOLUTA - NOVAS CORES
        tendencia_manha, slope_manha, r2_manha, var_abs_manha = calcular_tendencia(
            df_pivot['Data'], df_pivot['Manhã']
        )
        tendencia_tarde, slope_tarde, r2_tarde, var_abs_tarde = calcular_tendencia(
            df_pivot['Data'], df_pivot['Tarde']
        )
        
        # Formatar texto da variação com sinal
        sinal_manha = '+' if var_abs_manha >= 0 else ''
        sinal_tarde = '+' if var_abs_tarde >= 0 else ''
        
        ax.plot(df_pivot['Data'], tendencia_manha, 
                linestyle='--', linewidth=2, 
                label=f'Tendência Manhã ({sinal_manha}{var_abs_manha:.1f})', 
                color='#76B6FF', alpha=0.7, zorder=2)
        
        ax.plot(df_pivot['Data'], tendencia_tarde, 
                linestyle='--', linewidth=2, 
                label=f'Tendência Tarde ({sinal_tarde}{var_abs_tarde:.1f})', 
                color='#E67E22', alpha=0.7, zorder=2)
        
        # PADRONIZAR EIXO Y (0 até valor_max_global_individuais)
        ax.set_ylim(0, valor_max_global_individuais)
        
        # Configurar eixos e formatação
        ax.set_xlabel('Data', fontsize=12, fontweight='bold')
        ax.set_ylabel('Quantidade de Pessoas', fontsize=12, fontweight='bold')
        
        # Extrair nome curto do logradouro
        if ' - ' in logradouro:
            nome_log_curto = logradouro.split(' - ', 1)[1]
        else:
            nome_log_curto = logradouro
        
        ax.set_title(
            f'Evolução - {nome_log_curto}\n'
            f'Região: {regiao} | Período: {data_inicio_dt.strftime("%d/%m/%Y")} a {data_fim_dt.strftime("%d/%m/%Y")}',
            fontsize=14, fontweight='bold', pad=20
        )
        
        # Formatar eixo X com datas
        ax.xaxis.set_major_formatter(mdates.DateFormatter('%d/%m'))
        ax.xaxis.set_major_locator(mdates.DayLocator(interval=max(1, dias_periodo // 15)))
        plt.xticks(rotation=45, ha='right')
        
        # Grid
        ax.grid(True, alpha=0.3, linestyle='--', zorder=1)
        
        # Legenda CENTRALIZADA ABAIXO DO GRÁFICO
        ax.legend(loc='upper center', bbox_to_anchor=(0.5, -0.15), 
                 ncol=4, fontsize=10, framealpha=0.9)
        
        # Adicionar estatísticas no canto
        stats_text = (
            f'Manhã: Média={int(df_pivot["Manhã"].mean())} | '
            f'Max={int(df_pivot["Manhã"].max())} | '
            f'Min={int(df_pivot["Manhã"].min())}\n'
            f'Tarde: Média={int(df_pivot["Tarde"].mean())} | '
            f'Max={int(df_pivot["Tarde"].max())} | '
            f'Min={int(df_pivot["Tarde"].min())}'
        )
        ax.text(0.02, 0.98, stats_text, 
                transform=ax.transAxes, 
                fontsize=9, 
                verticalalignment='top',
                bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5))
        
        # Ajustar layout para acomodar a legenda
        plt.tight_layout()
        plt.subplots_adjust(bottom=0.15)
        
        # Salvar gráfico com nome limpo (sem caracteres problemáticos)
        nome_arquivo_limpo = limpar_nome_arquivo(nome_log_curto[:50])
        nome_arquivo = f"{regiao.replace(' ', '_')}_{idx:02d}_{nome_arquivo_limpo}.png"
        caminho_arquivo = pasta_graficos_individuais / nome_arquivo
        plt.savefig(caminho_arquivo, dpi=300, bbox_inches='tight')
        plt.close()
        
        total_graficos += 1
        print(f"  [{idx}/{len(logradouros_regiao)}] ✓ {nome_log_curto[:60]}")

print(f"\n{'=' * 80}")
print(f"✓ Total de gráficos individuais gerados: {total_graficos}")
print(f"{'=' * 80}")

# %% [markdown]
# ## 12.2. Gráficos de Médias por Região

# %%
print("\n" + "=" * 80)
print("GERANDO GRÁFICOS DE MÉDIAS POR REGIÃO")
print("=" * 80)

for regiao in regioes_ordenadas:
    print(f"\n📊 Gerando gráfico: {regiao}")
    
    # Pegar dados da região
    df_regiao = rankings_media[regiao].copy()
    
    # ARREDONDAR OS DADOS ANTES DE PLOTAR
    df_regiao['Manhã'] = df_regiao['Manhã'].round(0).astype(int)
    df_regiao['Tarde'] = df_regiao['Tarde'].round(0).astype(int)
    
    # Limitar aos top 15 para não poluir o gráfico (ou todos se forem poucos)
    max_logradouros = min(15, len(df_regiao))
    df_regiao_plot = df_regiao.head(max_logradouros)
    
    # Extrair nomes curtos dos logradouros (COMPLETOS)
    nomes_curtos = []
    for log in df_regiao_plot['Logradouro']:
        if ' - ' in log:
            nome_curto = log.split(' - ', 1)[1]
        else:
            nome_curto = log
        nomes_curtos.append(nome_curto)
    
    # Criar figura com altura ajustável
    fig, ax = plt.subplots(figsize=(16, max(10, max_logradouros * 0.6)))
    
    # Preparar dados
    x = np.arange(len(df_regiao_plot))
    width = 0.35
    
    # Criar barras agrupadas - NOVAS CORES
    bars1 = ax.barh(x - width/2, df_regiao_plot['Manhã'], width,
                    label='Manhã (10h)', color='#FF8C00', alpha=0.9)
    bars2 = ax.barh(x + width/2, df_regiao_plot['Tarde'], width,
                    label='Tarde (15h)', color='#1E90FF', alpha=0.9)
    
    # Configurar eixos
    ax.set_yticks(x)
    ax.set_yticklabels(nomes_curtos, fontsize=10)
    ax.set_xlabel('Média de Pessoas por Período', fontsize=12, fontweight='bold')
    ax.set_title(
        f'Média de Pessoas por Período - {regiao}\n'
        f'Período: {data_inicio_dt.strftime("%d/%m/%Y")} a {data_fim_dt.strftime("%d/%m/%Y")} '
        f'({dias_periodo} dias)',
        fontsize=14, fontweight='bold', pad=20
    )
    
    # PADRONIZAR EIXO X (0 até valor_max_global_regioes)
    ax.set_xlim(0, valor_max_global_regioes)
    
    # Adicionar rótulos de dados nas barras
    for bars in [bars1, bars2]:
        for bar in bars:
            width_val = bar.get_width()
            if width_val > 0:  # Só mostrar se houver valor
                ax.text(width_val, bar.get_y() + bar.get_height()/2,
                       f' {int(width_val)}',
                       va='center', fontsize=9, fontweight='bold')
    
    # Inverter eixo Y para ranking crescente
    ax.invert_yaxis()
    
    # Grid
    ax.grid(axis='x', alpha=0.3, linestyle='--')
    
    # Legenda CENTRALIZADA ABAIXO DO GRÁFICO
    ax.legend(loc='upper center', bbox_to_anchor=(0.5, -0.08), 
             ncol=2, fontsize=11, framealpha=0.9)
    
    # Ajustar layout para acomodar legenda (SEM NOTA)
    plt.tight_layout()
    plt.subplots_adjust(bottom=0.10)
    
    # Salvar gráfico
    nome_arquivo = f"media_periodos_{regiao.replace(' ', '_')}.png"
    caminho_arquivo = pasta_graficos_regioes / nome_arquivo
    plt.savefig(caminho_arquivo, dpi=300, bbox_inches='tight')
    plt.close()
    
    print(f"  ✓ Salvo: {nome_arquivo}")
    print(f"  ✓ Logradouros no gráfico: {max_logradouros}")

print(f"\n{'=' * 80}")
print(f"✓ Total de gráficos de região gerados: {len(regioes_ordenadas)}")
print(f"{'=' * 80}")

# %% [markdown]
# ## 12.3. Gráfico Consolidado - Visão Geral

# %%
print("\n" + "=" * 80)
print("GERANDO GRÁFICO CONSOLIDADO - VISÃO GERAL")
print("=" * 80)

# Pegar Top 10 geral
top10_geral = df_completo.nlargest(10, 'Média pessoas').copy()

# ARREDONDAR OS DADOS ANTES DE PLOTAR
top10_geral['Manhã'] = top10_geral['Manhã'].round(0).astype(int)
top10_geral['Tarde'] = top10_geral['Tarde'].round(0).astype(int)

# Criar figura
fig, ax = plt.subplots(figsize=(16, 12))

# Preparar dados - NOMES COMPLETOS
nomes_completos = []
for log in top10_geral['Logradouro']:
    if ' - ' in log:
        nome = log.split(' - ', 1)[1]  # Pegar só depois do " - "
    else:
        nome = log
    nomes_completos.append(nome)

x = np.arange(len(top10_geral))
width = 0.35

# Criar barras - NOVAS CORES
bars1 = ax.barh(x - width/2, top10_geral['Manhã'], width,
                label='Manhã (10h)', color='#FF8C00', alpha=0.9)
bars2 = ax.barh(x + width/2, top10_geral['Tarde'], width,
                label='Tarde (15h)', color='#1E90FF', alpha=0.9)

# Configurar
ax.set_yticks(x)
ax.set_yticklabels(nomes_completos, fontsize=11)
ax.set_xlabel('Média de Pessoas por Período', fontsize=13, fontweight='bold')
ax.set_title(
    f'Top 10 Logradouros - Média de Pessoas por Período (Consolidado)\n'
    f'Período: {data_inicio_dt.strftime("%d/%m/%Y")} a {data_fim_dt.strftime("%d/%m/%Y")} '
    f'({dias_periodo} dias, {total_periodos} períodos)',
    fontsize=15, fontweight='bold', pad=20
)

# Adicionar rótulos de dados nas barras
for bars in [bars1, bars2]:
    for bar in bars:
        width_val = bar.get_width()
        if width_val > 0:
            ax.text(width_val, bar.get_y() + bar.get_height()/2,
                   f' {int(width_val)}',
                   va='center', fontsize=10, fontweight='bold')

# Inverter eixo Y
ax.invert_yaxis()

# Grid
ax.grid(axis='x', alpha=0.3, linestyle='--')

# Legenda CENTRALIZADA ABAIXO DO GRÁFICO
ax.legend(loc='upper center', bbox_to_anchor=(0.5, -0.08), 
         ncol=2, fontsize=12, framealpha=0.9)

# Ajustar layout para acomodar legenda
plt.tight_layout()
plt.subplots_adjust(bottom=0.10)

# Salvar NO COMPLEXO_OKUHARA
caminho_consolidado = pasta_complexo / 'top10_consolidado_periodos.png'
plt.savefig(caminho_consolidado, dpi=300, bbox_inches='tight')
plt.close()

print(f"✓ Gráfico consolidado salvo: top10_consolidado_periodos.png")
print(f"✓ Localização: {pasta_complexo}")
print(f"{'=' * 80}")

# %% [markdown]
# ## 12.4. Resumo dos Gráficos Gerados

# %%
print("\n" + "=" * 80)
print("RESUMO - GRÁFICOS GERADOS")
print("=" * 80)

total_individuais = len(list(pasta_graficos_individuais.glob('*.png')))
total_regioes = len(list(pasta_graficos_regioes.glob('*.png')))
total_consolidado = len(list(pasta_complexo.glob('*.png')))

print(f"""
📊 GRÁFICOS DE EVOLUÇÃO TEMPORAL

ESTRUTURA DE PASTAS:
  📁 graficos/
      - complexo_okuhara/
          - top10_consolidado_periodos.png
          - logradouros_individuais/
          - regioes/

TOTAIS GERADOS:
  • Gráficos individuais (evolução temporal): {total_individuais}
    - Linhas de evolução diária (Manhã e Tarde)
    - Linhas de tendência com variação absoluta no eixo Y
    - Estatísticas (média, máx, mín)
    - Eixos padronizados (0 a {valor_max_global_individuais})
    - Rótulos com valores inteiros em todas as linhas
  
  • Gráficos por região (médias): {total_regioes}
    - Barras horizontais comparando Manhã vs Tarde
    - Top 15 logradouros de cada região
    - Nomes completos dos logradouros
    - Eixos padronizados (0 a {valor_max_global_regioes})
    - Rótulos com valores inteiros em todas as barras
  
  • Gráfico consolidado: {total_consolidado}
    - Top 10 geral com médias por período
    - Nomes completos dos logradouros
    - Rótulos com valores inteiros em todas as barras

TOTAL: {total_individuais + total_regioes + total_consolidado} gráficos

CARACTERÍSTICAS:
  ✓ Resolução: 300 DPI (alta qualidade)
  ✓ Período analisado: {data_inicio_dt.strftime('%d/%m/%Y')} a {data_fim_dt.strftime('%d/%m/%Y')}
  ✓ Total de dias: {dias_periodo}
  ✓ Total de períodos: {total_periodos}
  ✓ Linhas de tendência com variação absoluta (diferença no eixo Y)
  ✓ Estatísticas descritivas em cada gráfico individual
  ✓ Eixos padronizados (individuais e regiões)
  ✓ Valores inteiros em todos os rótulos de dados
  ✓ Legendas centralizadas abaixo dos gráficos
  ✓ Nomes completos dos logradouros em todos os gráficos
  ✓ Nomes de arqueres sem caracteres problemáticos

""")
print("=" * 80)
print("✓ GERAÇÃO DE GRÁFICOS CONCLUÍDA!")
print(f"✓ {datetime.now().strftime('%d/%m/%Y %H:%M:%S')}")
print("=" * 80)

✓ Bibliotecas importadas
✓ Análise iniciada em: 10/11/2025 13:05:51

DEFINIR PERÍODO DO RELATÓRIO

✓ Período selecionado: 01/10/2025 a 31/10/2025

Período: 01/10/2025 a 31/10/2025
  • Dias: 31
  • Períodos por dia: 2 (Manhã e Tarde)
  • Total de períodos: 62

SELEÇÃO DO ARQUIVO RAW
📂 Pasta raw: c:\Users\x504693\Documents\projetos\projeto_etl_dados\data\raw

📁 Arquivos disponíveis (mais recentes primeiro):
  1. Contagem_diaria_centro - Padronizada.xlsx
     Modificado em: 07/11/2025 18:00

  2. Contagem diária - Compilado.xlsx
     Modificado em: 07/11/2025 11:51

  3. CONTAGEM 2025 - CnR.xlsx
     Modificado em: 06/11/2025 12:37

  4. RelatorioCidadaoVinculado.xlsx
     Modificado em: 05/11/2025 16:38

  5. CidadaosVinculadosXBeneficios.xlsx
     Modificado em: 05/11/2025 10:42

  6. Base Okuhara Kohei.xlsx
     Modificado em: 21/10/2025 18:54

  7. nomes_Abordados.xlsx
     Modificado em: 21/10/2025 17:51

  8. Base Tablet.xlsx
     Modificado em: 21/10/2025 17:48

✓ Arquivo seleciona